In [148]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import time
import pandas as pd

In [174]:
def setup_driver():
    # Get the path for the ChromeDriver
    driver_path = ChromeDriverManager().install()

    # Set up Chrome options or capabilities (if needed)
    chrome_options = webdriver.ChromeOptions()
    # chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--disable-dev-shm-usage")

    # Create a Chrome service with the driver path
    chrome_service = Service(driver_path)

    # Initialize the Chrome WebDriver with options and service
    driver = webdriver.Chrome(service=chrome_service, options=chrome_options)
    return driver

def scrape_restaurant_links(driver, latitude, longitude, first_open):
    try:
        params = {
            "latitude": latitude,
            "longitude": longitude,
            "accuracy": 100
        }
        driver.execute_cdp_cmd("Emulation.setGeolocationOverride", params)

        driver.get("https://shop.ichefpos.com/explore")


        # ===Handle the dropbox message jump out===
        if first_open:
            # Wait until the dialog box is visible
            wait = WebDriverWait(driver, 30)
            dialog_box = wait.until(EC.visibility_of_element_located((By.XPATH, "//div[@role='dialog']")))

            # Find the "Share Location" button inside the dialog box
            share_location_button = dialog_box.find_element(By.XPATH, ".//button[contains(., '分享位置')]")
            share_location_button.click()
        
        # ===Search by current location===
        # Wait for the button to be clickable
        wait = WebDriverWait(driver, 10)
        button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".TakeoutFinderCircleButton__CircleButton-kxzgzF")))
        # Click the button
        button.click()

        # ===Search if there are no restaurant===
        time.sleep(2)
        wait = WebDriverWait(driver, 10)
        # Find the element by its class name
        no_restaurant_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.TakeoutFinderControls__NoRestaurantsInfo-lehtk.fPpujG")))
        # Get the 'style' attribute value
        style_str = no_restaurant_element.get_attribute('style')
        # Check if the 'opacity' property is set to 1
        if 'opacity: 1' in style_str:
            print('No restaurant found')
            shop_links = []
            shop_ids = []
            return shop_links, shop_ids
            
        # ===Find restaurant link===
        wait = WebDriverWait(driver, 10)
        # Wait for elements to load after clicking the button
        elements = wait.until(EC.visibility_of_all_elements_located((By.XPATH, "//*[contains(@id, 'restaurant-card')]")))

        shop_links = []
        shop_ids = []
        # Loop through each element to find anchor tags and extract href attribute
        for element in elements:
            anchor_tags = element.find_elements(By.TAG_NAME, 'a')
            for anchor_tag in anchor_tags:
                href = anchor_tag.get_attribute("href")
                id = anchor_tag.get_attribute("id")
                shop_links.append(href)
                shop_ids.append(id)
        return shop_links, shop_ids

    except Exception as e:
        print(f'An error occurred: {str(e)}')
        return [], []

In [175]:
def store_restaurant_data(driver, latitude, longitude, first_open, filename='restaurant_data.json'):
    # Get the restaurant links and IDs
    restaurant_links, restaurant_ids = scrape_restaurant_links(driver, latitude, longitude, first_open)
    print(f'{len(restaurant_ids)} shops')

    # Create a list of dictionaries to store the data
    restaurant_data = []
    for link, id in zip(restaurant_links, restaurant_ids):
        restaurant_data.append({'id': id, 'link': link})

    # Load existing data from the file (if any)
    try:
        with open(filename, 'r') as file:
            existing_data = json.load(file)
    except FileNotFoundError:
        existing_data = []

    # Append the new data to the existing data
    existing_data.extend(restaurant_data)

    # Write the updated data back to the file
    with open(filename, 'w') as file:
        json.dump(existing_data, file, indent=4)


In [187]:
def create_coords(row, step = 0.0035):
    north, south, east, west = row['north'], row['south'], row['east'], row['west']
    coords = []
    current_lat = north
    while current_lat >= south:
        current_lon = west
        while current_lon <= east:
            coords.append([current_lat, current_lon])
            current_lon += step
        current_lat -= step
    return coords

In [188]:
# ===Set search area and coordinates===
central_df = pd.read_csv('fp_points.csv')
# Create the corner coordinates of the search rectangle
central_df['north'] = central_df['newLat'] + 0.001982
central_df['south'] = central_df['newLat'] - 0.001982
central_df['east'] = central_df['newLng'] + 0.02161
central_df['west'] = central_df['newLng'] - 0.02161
central_df['coords'] = central_df.apply(create_coords, axis=1)
central_df.head()

,newLat,newLng,geometry,Unnamed: 3,is_fp_point,north,south,east,west,coords
0,21.941414,120.720446,c(120.720446352,21.9414135330001),1,21.943396,21.939432,120.742056,120.698836,"[[21.943395533000103, 120.698836352], [21.9433..."
1,21.941414,120.800461,c(120.800460992,21.9414135330001),1,21.943396,21.939432,120.822071,120.778851,"[[21.943395533000103, 120.778850992], [21.9433..."
2,21.981448,120.720446,c(120.720446352,21.9814483630001),1,21.983430,21.979466,120.742056,120.698836,"[[21.983430363000103, 120.698836352], [21.9834..."
3,21.981448,120.760454,c(120.760453672,21.9814483630001),1,21.983430,21.979466,120.782064,120.738844,"[[21.983430363000103, 120.738843672], [21.9834..."
4,22.021483,120.720446,c(120.720446352,22.0214831930001),1,22.023465,22.019501,120.742056,120.698836,"[[22.023465193000103, 120.698836352], [22.0234..."


緯度1度 ＝ 111 km
1 km = 0.009 度
4.4 km = 0.0396 度
-> 北方=中心點往上 0.0396/2 (0.01982)度
-> 南方=中心點往下 0.0396/2

經度1度 ＝ 111*cos𝜃 = 101.794km 𝜃=23.5
1km = 0.0098度
4.4km = 0.0432度
-> 東方=中心點往右 0.0432/2 (0.02161)度
-> 西方=中心點往左 0.0432/2

In [190]:
# ===Run functions===
driver = setup_driver()
first_open = True
for row in range(390, central_df.shape[0]):
    print(f'第{row}個中心點')
    coords = central_df['coords'][row]
    for coord in coords:
        lat = coord[0]
        long = coord[1]
        store_restaurant_data(driver, lat, long, first_open)
        first_open = False
        time.sleep(1)
driver.quit()

第390個中心點
No restaurant found
0 shops
No restaurant found
0 shops
No restaurant found
0 shops
No restaurant found
0 shops
No restaurant found
0 shops
No restaurant found
0 shops
No restaurant found
0 shops
No restaurant found
0 shops
6 shops
8 shops
3 shops
4 shops
4 shops
No restaurant found
0 shops
No restaurant found
0 shops
No restaurant found
0 shops
1 shops
An error occurred: Message: 

0 shops
No restaurant found
0 shops
No restaurant found
0 shops
6 shops
7 shops
7 shops
11 shops
4 shops
3 shops
第391個中心點
4 shops
4 shops
2 shops
No restaurant found
0 shops
2 shops
4 shops
4 shops
4 shops
14 shops
18 shops
24 shops
26 shops
22 shops
4 shops
3 shops
1 shops
No restaurant found
0 shops
No restaurant found
0 shops
3 shops
5 shops
8 shops
14 shops
19 shops
24 shops
23 shops
19 shops
第392個中心點
23 shops
16 shops
16 shops
15 shops
18 shops
35 shops
49 shops
47 shops
50 shops
51 shops
35 shops
30 shops
26 shops
22 shops
16 shops
18 shops
22 shops
27 shops
52 shops
65 shops
75 shops
77 shop

KeyboardInterrupt: 